In [1]:
import os

In [5]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer\\research'

In [6]:
os.chdir("../")

In [7]:
pwd

'd:\\PW_DS\\NLP\\End-To-End-Text-Summarizer-Project-using-Transformer'

## Entity??

In [30]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



## Update src condfig manager

In [15]:
from Text_Summarizer.constants import *
from Text_Summarizer.utils.common import read_yaml,create_directories

In [17]:
CONFIG_FILE_PATH
PARAMS_FILE_PATH

WindowsPath('params.yaml')

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        


    def get_data_ingestion_config(self)-> DataIngestionConfig: 
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir,
            )

            return data_ingestion_config



## Update Components

In [19]:
import os
import urllib.request as request
import zipfile
from Text_Summarizer.logging import  logger
from Text_Summarizer.utils.common import get_size

In [42]:
class DataIngestion:
    def __init__(
        self,
        config: DataIngestionConfig):
    
        self.config = config


    def download_file(self):
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )

                logger.info(f"{filename} download! with following info: \n {headers}")

            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
            """
            zip_file_pah: str
            Extarcts the zip file into the  data directory 
            Function return None
            """
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            # logger.info(f"{self.config.local_data_file}")
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"{self.config.local_data_file} unzipped to {unzip_path}")  


## Update the Pipeline

In [46]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 

[2024-01-10 17:39:48,373: INFO: common: yaml file: config\config.yaml loaed successfully.]
[2024-01-10 17:39:48,376: INFO: common: yaml file: params.yaml loaed successfully.]
[2024-01-10 17:39:48,377: INFO: common: directory: artifacts created successfully.]
[2024-01-10 17:39:48,380: INFO: common: directory: artifacts/data_ingestion created successfully.]
[2024-01-10 17:39:52,394: INFO: 3441075174: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6BE0:17E986:125E20:1AB9FA:659E890C
Accept-Ranges: bytes
Date: Wed, 10 Jan 2024 12:09:49 GMT
Via: 1.1 varnish
X-Served-By: cache-